In [ ]:
import numpy as np

from skorecard import datasets
from skorecard.bucketing.bucketing import SimpleBucketTransformer
from skorecard.bucketing.bucketing import AgglomerativeBucketTransformer
from skorecard.bucketing.bucketing import QuantileBucketTransformer

In [ ]:
df = datasets.load_uci_credit_card(as_frame=True)
df.head(10)


### Simple Example - Apply same bucket size to all features

In [ ]:
X = df[['LIMIT_BAL', 'BILL_AMT1']].values
MyBucketTransformer = SimpleBucketTransformer(bin_count=5)

In [ ]:
MyBucketTransformer.fit(X)

In [ ]:
MyBucketTransformer.transform(X)

### Using a dictionary for different bucket sizes

In [ ]:
X = df[['LIMIT_BAL', 'BILL_AMT1']].values

In [ ]:
MyBucketTransformer = SimpleBucketTransformer(bin_count=[5, 7])

In [ ]:
MyBucketTransformer.fit(X)

In [ ]:
MyBucketTransformer.transform(X)

### Practical Example with Pipelines

In [ ]:
from sklearn.compose import ColumnTransformer
from skorecard.bucketing.bucketing import ManualBucketTransformer

In [ ]:
df = datasets.load_uci_credit_card(as_frame=True)
df.head(10)

In [ ]:
# define transform
transformer = ColumnTransformer(
    transformers=[
        ('simple', SimpleBucketTransformer(bin_count=2), [1]),
        ('agglom', AgglomerativeBucketTransformer(bin_count=4), [0]),
        ('quantile', QuantileBucketTransformer(bin_count=[10, 6]), [2,3])
    ]
)

In [ ]:
transformer.fit_transform(df[['EDUCATION', 'MARRIAGE', 'LIMIT_BAL', 'BILL_AMT1']].values)

In [ ]:
transformer.named_transformers_.quantile.BucketDict

In [ ]:
boundary_dict = {}

for i in range(len(transformer.named_transformers_)):
    key = list(transformer.named_transformers_.keys())[i]
    for j in range(len(transformer.transformers[i][2])):
        boundary_dict[transformer.transformers[i][2][j]]= transformer.named_transformers_[key].BucketDict[f'Feature_{j}'].boundaries

In [ ]:
MBT = ManualBucketTransformer(boundary_dict=boundary_dict)

In [ ]:
X = df[['EDUCATION', 'MARRIAGE', 'LIMIT_BAL', 'BILL_AMT1']].values
MBT.fit(X)
MBT.transform(X)